In [24]:
import mlflow

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
#import mlflow.xgboost


In [3]:
mlflow.set_tracking_uri("http://localhost:5002/")
mlflow.autolog()

exp_name = "SL-XGBoost-Test1"
artifact_loc= "/Users/vijaya/DS_workspace/MLflow/mlruns"
try:
    experiment = mlflow.get_experiment_by_name(exp_name)
    experiment_id = experiment.experiment_id
except AttributeError:
    experiment_id = mlflow.create_experiment(name=exp_name,artifact_location=artifact_loc)
mlflow.set_experiment(exp_name)

2023/06/09 16:28:43 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


<Experiment: artifact_location='/Users/vijaya/DS_workspace/MLflow/mlruns', creation_time=1686308323509, experiment_id='189366309187576494', last_update_time=1686308323509, lifecycle_stage='active', name='SL-XGBoost-Test1', tags={}>

In [15]:
exp = mlflow.get_experiment(experiment_id)

print(f"Name: {exp.name}")
print(f"Artifact Location: {exp.artifact_location}")
print(f"Experiment-id: {exp.experiment_id}")
print(f"Lifecycle-stage: {exp.lifecycle_stage}")

Name: SL-XGBoost-Test1
Artifact Location: /Users/vijaya/DS_workspace/MLflow/mlruns
Experiment-id: 189366309187576494
Lifecycle-stage: active


# Building model

In [29]:
# 1. loading data
source = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/winequality-white.csv"
data=pd.read_csv(source)

# 2. splitting data into train, test
train, test = train_test_split(data, test_size=0.2, random_state=42)
x_train = train.drop("quality", axis=1)
x_test = test.drop("quality", axis=1)

y_train = train["quality"]
y_test = test["quality"]

# Define model and model metrics tracking

In [33]:
with mlflow.start_run(experiment_id=exp.experiment_id, run_name='run_3') as run:
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id
    
    print("MLFlow")
    print("RUN ID:", run_id)
    print("Experiment ID:", experiment_id)
    print("Experiment Name:", mlflow.get_experiment(experiment_id).name)
    
    
    #model parameters
    params = {
            'max_depth':7,
            'min_child_weight':1,
            'random_state':2022
            }

    
    # MLflow parameters
    print("MLflow Parameters:")
    print(" - max_depth:", params['max_depth'])
    print(" - min_child_weight:", params['min_child_weight'])
    
    model = xgb.XGBRegressor(**params)
    model.fit(x_train, y_train)
    
     # MLflow Artifacts
    prediction = model.predict(x_test)
    #print("Prediction:", prediction)
    rmse = np.sqrt(mean_squared_error(y_test, prediction))
    mse = mean_squared_error(y_test, prediction)
    r2 = r2_score(y_test, prediction)
    print("Metrics :")
    print("- RMSE:", rmse)
    print("- MSE:", mse)
    print("- R2:", r2)
    
    
    # MLflow metrics
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("MaxDepth", params['max_depth'])
    mlflow.log_metric("MinChildWeight", params['min_child_weight'])
    mlflow.log_metric("RandomState", params['random_state'])
    

2023/06/09 17:52:07 WARNING mlflow.xgboost: Failed to log dataset information to MLflow Tracking. Reason: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 8ef4423f63f44973ad47f8b1ab056986. Failed operations: [MlflowException("API request to endpoint /api/2.0/mlflow/runs/log-inputs failed with error code 404 != 200. Response body: \'<!doctype html>\\n<html lang=en>\\n<title>404 Not Found</title>\\n<h1>Not Found</h1>\\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\\n\'")]')]


MLFlow
RUN ID: 8ef4423f63f44973ad47f8b1ab056986
Experiment ID: 189366309187576494
Experiment Name: SL-XGBoost-Test1
MLflow Parameters:
 - max_depth: 7
 - min_child_weight: 1


2023/06/09 17:52:08 WARNING mlflow.sklearn: Failed to log evaluation dataset information to MLflow Tracking. Reason: API request to endpoint /api/2.0/mlflow/runs/log-inputs failed with error code 404 != 200. Response body: '<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'


Metrics :
- RMSE: 0.6207989684298344
- MSE: 0.3853913592035465
- R2: 0.5023825344054536


# Model Register Individually

In [34]:
#log MLflow model
mlflow.xgboost.log_model(model, "artifact", registered_model_name="SL-XGBoost-Test1")

Successfully registered model 'SL-XGBoost-Test1'.
2023/06/09 17:57:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: SL-XGBoost-Test1, version 1
Created version '1' of model 'SL-XGBoost-Test1'.


# Model Predictions using Registered Model

In [ ]:
#predict with MLFlow model(SL-XGBoost-Test1)
model = mlflow.xgboost.load_model('models:///XGBoost_Model/Production')
print("Model: \n", model)
print("="*50)
print("Model:\n", model)
print("="*50)
prediction = model.predict(X_test)
# print("Prediction.type:", type(prediction))
# print("="*50)
# print("Prediction.shape:", prediction.shape)
# print("="*50)
print("Prediction:\n", prediction)
print("Prediction Done.")